In [1]:
from sodapy import Socrata
import pandas as pd
import mysql.connector as mysql

In [2]:
#function to connect ot mysql 
def connect_db(db_name , host , user , password):
    try:
        db = mysql.connect(
            host = host,
            user = user,
            password = password,
            database = db_name
        )
        
        return db
    except:
        print("could not access the database")

In [3]:
def create_tables(cursor):
    cursor.execute("SHOW TABLES")
    current_tables = list(cursor)
    current_tables = list(map(lambda x:x[0] , current_tables))
    if "Sensors" not in current_tables:
        Q1 = '''CREATE TABLE Sensors(sensor_id INT PRIMARY KEY, 
            sensor_description VARCHAR(100), 
            sensor_lat FLOAT, 
            sensor_long FLOAT)
            '''
        
        cursor.execute(Q1)
        print("Sensors table created")
    else:
        print("Sensors table already exists")
        
    if "Sensor_data" not in current_tables:
        Q2 = '''CREATE TABLE Sensor_data(id int PRIMARY KEY AUTO_INCREMENT ,
        sensor_id INT , 
        value INT , 
        date_time DATETIME )'''
        
        Q3 = "ALTER TABLE Sensor_data ADD FOREIGN KEY(sensor_id) REFERENCES Sensors(sensor_id)"
        cursor.execute(Q2)
        cursor.execute(Q3)
        print("sensor_data table created")
    else : 
        print("sensor_data already exists")

In [4]:
def populate_sensors(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC" ):
    client = Socrata("data.melbourne.vic.gov.au", access_token)
    results = client.get("h57g-5234")
    results_df = pd.DataFrame.from_records(results)
    sensors = results_df[["sensor_id" , "sensor_description" , "latitude" , "longitude"]]
    # sensors.iloc[:,0] = sensors.sensor_id.astype("int")
    # sensors.iloc[:,1] = sensors.sensor_description.astype("str")
    # sensors.iloc[:,2] = sensors.latitude.astype("float")
    # sensors.iloc[:,3] = sensors.longitude.astype("float")
    sensors_list = [tuple(x) for x in sensors.to_numpy()]

    Q4 = "INSERT INTO Sensors(sensor_id , sensor_description, sensor_lat , sensor_long) VALUES (%s , %s , %s , %s)"
    cursor.executemany(Q4 , sensors_list)
    print("successfully added data to sensors table ")
    return 1

In [5]:
def populate_sensors_data(cursor , access_token = "RrWl8vIygHuQlviEbWq1TD5zC"):
    client = Socrata("data.melbourne.vic.gov.au",access_token)
    results = client.get("d6mv-s43h")
    results_df = pd.DataFrame.from_records(results)
    sensor_data = results_df[["sensor_id"  , "total_of_directions" , "date_time"]]
    sensors_data_list = [tuple(x) for x in sensor_data.to_numpy()]
    
    Q5 = "INSERT INTO Sensor_data(sensor_id , value , date_time) VALUES (%s, %s , %s)"
    cursor.executemany(Q5 , sensors_data_list)
    print("successfully added data to sensors_data table")

In [8]:
# db = connect_db("OWLLEAD_DB" , "owllead-db1.ckoizwonzrvu.us-east-2.rds.amazonaws.com" , "admin", "owllead999")
cursor = db.cursor()
# create_tables(cursor)
# populate_sensors(cursor)
# populate_sensors_data(cursor)

In [15]:
cursor.execute('''SELECT  Sensors.sensor_description , AVG(Sensor_data.value) as avg_count
FROM Sensor_data
INNER JOIN Sensors on Sensor_data.sensor_id = Sensors.sensor_id
GROUP BY Sensors.sensor_description 
ORDER BY avg_count DESC''')
ans = list(cursor)


[('QVM-Franklin St (North)', Decimal('3.0000')),
 ('Elizabeth St-La Trobe St (East)', Decimal('2.8158')),
 ('Waterfront City', Decimal('2.7500')),
 ('Melbourne Central-Elizabeth St (East)', Decimal('2.7045')),
 ('Bourke St-Russell St (West)', Decimal('2.3462')),
 ('Melbourne Central', Decimal('2.1429')),
 ('Princes Bridge', Decimal('2.1053')),
 ('Spencer St-Collins St (North)', Decimal('2.0769')),
 ('Webb Bridge', Decimal('2.0000')),
 ('Bourke St Bridge', Decimal('2.0000')),
 ('QV Market-Peel St', Decimal('2.0000')),
 ('Flinders St-Elizabeth St (East)', Decimal('2.0000')),
 ('The Arts Centre', Decimal('1.9000')),
 ('QVM-Therry St (South)', Decimal('1.8667')),
 ('Swanston St - City Square', Decimal('1.8333')),
 ('Flinders La-Swanston St (West)', Decimal('1.8077')),
 ('QVM-Queen St (East)', Decimal('1.7500')),
 ('Flinders St- ACMI', Decimal('1.7500')),
 ('RMIT Building 14', Decimal('1.6842')),
 ('Bourke St - Spencer St (South)', Decimal('1.6667')),
 ('Pelham St (South)', Decimal('1.6667'

In [17]:
cursor.execute("SELECT * from Sensors")
list(cursor)

[(1, 'Bourke Street Mall (North)', -37.8135, 144.965),
 (2, 'Bourke Street Mall (South)', -37.8138, 144.965),
 (3, 'Melbourne Central', -37.811, 144.964),
 (4, 'Town Hall (West)', -37.8149, 144.966),
 (5, 'Princes Bridge', -37.8187, 144.968),
 (6, 'Flinders Street Station Underpass', -37.8191, 144.966),
 (7, 'Birrarung Marr', -37.8186, 144.972),
 (8, 'Webb Bridge', -37.8229, 144.947),
 (9, 'Southern Cross Station', -37.8198, 144.951),
 (10, 'Victoria Point', -37.8188, 144.947),
 (11, 'Waterfront City', -37.8157, 144.94),
 (12, 'New Quay', -37.8146, 144.943),
 (13, 'Flagstaff Station', -37.8124, 144.957),
 (14, 'Sandridge Bridge', -37.8201, 144.963),
 (15, 'State Library', -37.8106, 144.964),
 (16, 'Australia on Collins', -37.8157, 144.965),
 (17, 'Collins Place (South)', -37.8136, 144.973),
 (18, 'Collins Place (North)', -37.8134, 144.973),
 (19, 'Chinatown-Swanston St (North)', -37.8124, 144.965),
 (20, 'Chinatown-Lt Bourke St (South)', -37.8117, 144.968),
 (21, 'Bourke St-Russell St 